<a href="https://colab.research.google.com/github/curiosity806/machine-learning-and-practice-report/blob/master/Dacon_satellite_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [36]:
# 훈련 데이터
!cp '/content/drive/My Drive/2020 Kaggle Study/data/train_final.npy' train.npy

In [37]:
import numpy as np

train = np.load('train.npy')

In [38]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import random
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Conv2DTranspose, MaxPooling2D, BatchNormalization, Activation, concatenate, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
import warnings
import gc
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy import stats
from mpl_toolkits.axes_grid1 import make_axes_locatable

warnings.filterwarnings("ignore")

# 재생산성을 위해 시드 고정
np.random.seed(7)
random.seed(7)
tf.random.set_seed(7)

# 모델링

In [40]:
def mae(y_true, y_pred):

    y_true = np.array(y_true)
    y_true = y_true.reshape(1, -1)[0]  
    
    y_pred = np.array(y_pred)
    y_pred = y_pred.reshape(1, -1)[0]
    
    mae = np.mean(np.abs(y_true - y_pred))
    
    return mae

In [59]:
train.shape

(11905, 15)

In [60]:
X = train[:,:14]
y = train[:,-1]
X.shape, y.shape

((11905, 14), (11905,))

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## 선형회귀

In [62]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [63]:
pred = lr.predict(X_test)

In [64]:
mae(y_test, pred)

19.323816

## 다항회귀

In [65]:
from sklearn.preprocessing import PolynomialFeatures

interaction = PolynomialFeatures(degree=3, include_bias=False, interaction_only=True)
features_interaction = interaction.fit_transform(X_train)

In [67]:
from sklearn.linear_model import LinearRegression
test_interaction = interaction.transform(X_test)
lr_n = LinearRegression(normalize=True)
lr_n.fit(features_interaction, y_train)
pred_n = lr_n.predict(test_interaction)

In [68]:
mae(y_test, pred_n)

19.789162

In [69]:
lr = LinearRegression()
lr.fit(features_interaction, y_train)

pred = lr.predict(test_interaction)

In [70]:
mae(y_test, pred)

20.080462

## SVM

In [ ]:
from sklearn.svm import SVR
svm_inst = SVR(kernel='linear', verbose = True)
svm_inst.fit(X_train, y_train)

[LibSVM]

In [54]:
pred = svm_inst.predict(X_test)

In [55]:
mae(y_test, pred)

0.07594276934915517

In [58]:
for i in range(10):
    print(y_test[10*i], pred[10*i])

24.125925 24.21571588724777
63.896336 63.95300706896425
68.18409 68.24608697592814
21.14196 21.231008991900072
28.215694 28.301503451104683
121.82484 121.84499279237568
100.93867 100.9656434676653
123.45101 123.46762215823082
117.79041 117.80816625983584
68.229294 68.28089956351576


## XGBoost

In [ ]:
import time
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier, XGBModel
from xgboost import plot_importance, plot_tree


# create a default XGBoost classifier
model = XGBClassifier(n_estimators=500, random_state=seed)

# create a default XGBoost classifier
model = XGBClassifier(
    random_state=seed, 
    eval_metric=["error", "auc"]
)
# Create the grid search parameter grid and scoring funcitons
param_grid = {
    "colsample_bytree": [0.6, 0.8, 1.0],
    "subsample": [0.6, 0.8, 1.0],
    "max_depth": [2, 3, 4],
    "n_estimators": [100, 200, 300, 400],
    "reg_lambda": [1, 1.5, 2],
    "gamma": [0, 0.1, 0.3],
}
scoring = {
    'AUC': 'roc_auc', 
    'Accuracy': make_scorer(accuracy_score)
}
# create the Kfold object
num_folds = 10
kfold = StratifiedKFold(n_splits=num_folds, random_state=1)
# create the grid search object
n_iter=50
grid = RandomizedSearchCV(
    estimator=model, 
    param_distributions=param_grid,
    cv=kfold,
    scoring=scoring,
    n_jobs=-1,
    n_iter=n_iter,
    refit="AUC",
)
# fit grid search
%time best_model = grid.fit(X_train, y_train)

# PCA

In [ ]:
from sklearn.decomposition import PCA
